# Text Classification with fastText

This quick tutorial introduces the task of text classification using the [fastText](https://fasttext.cc/) library and tries to show what the full pipeline looks like from the beginning (obtaining the dataset and preparing the train/valid split) to the end (predicting labels for unseen input data).

Greatly based on a previously made [tutorial](https://colab.research.google.com/github/NaiveNeuron/nlp-excercises/blob/master/tutorial2-fasttext/Text_Classification_fastText.ipynb)

## The Cooking StackExchange tags dataset

We'll use a dataset of a few thousand questions asked on [Cooking StackExchange](https://cooking.stackexchange.com/) which have various tags assigned to them and which already exists in the fastText format -- basically a text file where each line contains one text document that is to be classified. Note that the lines start with `__label__` tags which denote the "ground truth" label for that particular text document.


`__label__<X> __label__<Y> ... <Text>`


For example:

`__label__chocolate American equivalent for British chocolate terms`

--------------------------

In the next few cells we'll download the dataset and take a closer look at what the data looks like (using the [`head`](https://linux.101hacks.com/unix/head/) command) and some further statistics about the dataset (using the [`wc`](https://www.tecmint.com/wc-command-examples/) -- command).


In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

In [0]:
!head cooking.stackexchange.txt

In [0]:
!wc cooking.stackexchange.txt 

We've got roughly 15k samples in our dataset. Let's split it into a training set of roughly 12k samples and testing set of 3k samples.

In [0]:
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

In [0]:
%ls

## Installation of fastText

Installing fastText is realtively easy on any Unix-like system -- running the following cell should be enough to build the `fasttext` binary, which is all we need in this tutorial.

In [0]:
!git clone https://github.com/facebookresearch/fastText.git

In [0]:
%cd fastText
!make

In [0]:
!cp fasttext ../
%cd ..

## Training and testing a fastText model

The actual model fastText implements is rather simple.

You can find more details on the model in section 2 the introductory paper: [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759). Note that the document representation $x_n$ is computed as the average of the embedding of all document features (words, word ngrams or character ngrams -- more on that later).


![The actual model architecture of fastText classification](https://d3i71xaburhd42.cloudfront.net/9d6993f60539d30ee325138b3465aa020fa3bcb4/6-Figure2-1.png)


In the following cell we run the `supervised` command which trains a fastText model using the data in `./cooking.train` and saves the model to `./cooking_model1`.

In [0]:
!./fasttext supervised -input ./cooking.train -output ./cooking_model1

### Accuracy

Still, looking at just the summary statistics is not really that much fun -- that usually comes from trying the model out on some real-world data. We can easily do that with fastText by running something like the command in the following cell:

Now let's see how the model does on the validation set.

In [0]:
!./fasttext test cooking_model1.bin ./cooking.valid 3

Looking at the results, they do not look very stellar, as both the $P@3$ and $R@3$ can be values from 0 to 1. But what do those $P@3$ and $R@3$ actually represent?

The short answer is that they are the average [precision](https://en.wikipedia.org/wiki/Precision_and_recall#Precision) and [recall](https://en.wikipedia.org/wiki/Precision_and_recall#Recall) averaged over all test examples. Plus, the $@3$ tell us that we only consider the top 3 most probable labels predicted by the system. 



**Python** version of training and testing the accuracy: https://github.com/facebookresearch/fastText/blob/master/python/doc/examples/train_supervised.py

### Prediction 
That probably did not help too much, so let's try to illustrate what that would mean using the following example:

`__label__storage-method __label__equipment __label__bread __label__food-safety What's the purpose of a bread box?`

In [0]:
!echo "What's the purpose of a bread box?" | ./fasttext predict-prob ./cooking_model1.bin - 3



As we can see, our data shows that for "*What's the purpose of a bread box?*" the model is supposed to predict the following labels with high probability: 

- `storage-method`
- `equipment`
- `bread`
- `food-safety`

Suppose, however, that our model predicts the following labels as the top 3 most probable:

- `bread`
- `equipment`
- `box`

Since two labels (`bread` and `equipment`) out of three were found in the top 3 most probable predictions of the model, the $P@3$ in this particular example would be $P@3 = \frac{2}{3} = 0.66 $. We can interpret that as the model *precisely predicting* $\frac{2}{3}$ of the top 3 predictions it provided.

Furthermore, since two labels (again `bread` and `equipment`) out of all 4 that were denoted in the data as correct (or relevant) could be found in the top 3 most probable predictions of the model, the $R@3$ would be $R@3 = \frac{2}{4} = 0.5 $. That in turn can be interpreted as the model being able to *recall* $\frac{2}{4}$ of the correct (or relevant) labels in its top 3 most probable predictions.

As we can see, the reported performance and recall of our models will strongly depend on how many of the top most probable predictions should be considered. Also, we need to consider our final use case when deciding which of these to optimize for -- in the Cooking StackExchange example we may be more interested in recall than in precision.

--------------------------------


### Arguments
Armed with this understanding of the metrics we can try to improve, let's see what options does fastText allow us to set and see if we can get the model to perform better.

In [0]:
!./fasttext supervised

There are a couple of interesting options we'll dive a bit deeper into:

### Character ngrams (`minx` and `maxn`)

One of the interesting things fastText is capable of doing is incorporating character level information when preparing word vectors. You can find all the glory details in the [Enriching Word Vectors with Subword Information](https://arxiv.org/abs/1607.04606) paper, but the basic idea is as follows:

Given the word `banana` and $n=3$, fastText would generate the following ngrams:

- `<ba`
- `ban`
- `ana`
- `nan`
- `ana`
- `na>`

where the `<` and `>` represent the beginning and end of the word, respectively. That is quite useful because if we also had the word *ban* as part of the vocabulary, it would be represented as `<ban>` which makes it distinguishable from `ban` we extracted from banana.

Note that we are still talking about bag of words model and thus only the presence of a respective ngram matters. Still, thanks to this nice setup we are pretty much by default able to model prefixes and suffixes. That is of huge practical value, since even if we now encountered say the word `bananoid`  which was not present in training data, thanks to the aforementioned character ngrams we are able to assign it at least some representation, rather than calling it an unknown word and replacing its occurences with `UNK`, which is what the standard approach would be.

In fastText the length of ngrams can be set via the `-minn` and `-maxn` flags, which control the minimum and maximum length of ngrams fastText considers. By default these are set to 0, which basically turns this feature off.

Let's see if our `bananoid` example would actually work by saving the word vectors fastText produces during training and trying to find out which words are the closest neighbors of `bananoid` in the learned vector space

In [0]:
!./fasttext supervised -minn 3 -maxn 5 -input ./cooking.train -output ./cooking_model1 -saveOutput

In [0]:
!echo "bananoid" | ./fasttext nn ./cooking_model1.bin

Although it is obvious that our training data leave a lot to be desired, seeing `bananoid` close to `bananas`, `bananas?` and `banana?` seems to suggest that the ngrams do indeed have the potential to help with out of vocabulary words.

### Word ngrams

Similarly to character ngrams, fastText can also generate ngrams from words in the document. This can be set using the `-wordNgrams` flag which is set to 1 by default: only unigrams (single words) are considered. When we set it to say 2, the sentece `smash all potatoes` would be represented as

- `<smash>`
- `<all>`
- `<potatoes>`
- `<smash all>`
- `<all potatoes`

-----------------

Using these and some of the other available options, let us train a new version of the model and see how it performs.


In [0]:
!./fasttext supervised -minCount 2 -wordNgrams 3 -minn 3 -maxn 8 -lr 0.7 -dim 100 -epoch 25 -input ./cooking.train -output cooking_model2

In [0]:
!./fasttext test cooking_model2.bin ./cooking.valid 3

Trying the model out on some real-world data

In [0]:
!echo "Does it make sense to cook smashed potatoes?" | ./fasttext predict-prob ./cooking_model2.bin - 1

Alternatively we can also ask for more than just the most probable label:

In [0]:
!echo "Does it make sense to cook smashed potatoes?" | ./fasttext predict-prob ./cooking_model2.bin - 3

Or ask for as many predictions as possible (`-1`) but only taking into account those that have probability higher than `0.02`:

In [0]:
!echo "Does it make sense to cook smashed potatoes?" | ./fasttext predict-prob ./cooking_model2.bin - -1 0.02

**Python version** for predicting the labels can be found here: https://github.com/facebookresearch/fastText/tree/master/python

# **Automatic hyperparameter optimization**


As we saw in the tutorial, finding the best hyperparameters is crucial for building efficient models. However, searching the best hyperparameters manually is difficult. Parameters are dependent and the effect of each parameter vary from one dataset to another.


FastText's autotune feature allows you to find automatically the best hyperparameters for your dataset.

For more details, refer to: https://fasttext.cc/docs/en/autotune.html

### Basic Data Cleaning

In [0]:
!cat cooking.stackexchange.txt | tr -d "\([;.\!?,'/()]\)/" |  tr -d [:digit:] | tr "[:upper:]" "[:lower:]" > cooking.stackexchange.cleaned

In [0]:
!head -n 13404 cooking.stackexchange.cleaned > cooking_at.train
!tail -n 2000 cooking.stackexchange.cleaned > cooking_at.valid

In [0]:
!./fasttext supervised -autotune-validation cooking_at.valid -autotune-duration 600 -input cooking_at.train -output cooking_model_at


In [0]:
!./fasttext test cooking_model_at.bin ./cooking_at.valid 3

### English word vectors

https://fasttext.cc/docs/en/english-vectors.html lists various pre-trained word vectors. Let's look at **wiki-news-300d-1M.vec.zip** which contains 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens). We will play around with the vectors in python

In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.bin.zip 

In [0]:
!unzip wiki-news-300d-1M-subword.bin.zip

In [0]:
from fasttext import load_model

model = load_model('wiki-news-300d-1M-subword.bin')

As we saw before, the model has not seen the word canad but can provide an embedding for it using the subword information.

In [0]:
model.get_nearest_neighbors('canad')

In [0]:
model['canad']

We can play around with word analogies. For example, we can see if our model can guess what is to France, what Berlin is to Germany.




In [0]:
model.get_analogies("berlin", "germany", "france")